# Lesson 5 – Solutions Notebook  
Looping Pattern: Social Post Draft Refinement Agent

This notebook contains **one possible solution** for the exercises in:

`05_looping_social_post_refinement.ipynb`

Your own solutions may differ but still be valid as long as they implement the desired behavior.


## 1. Imports and Extended State Definition

We extend `PostState` to include:

- `stop_reason: str` – why the loop ended.
- `feedback: str` – user preference like `"shorter"` or `"more examples"`.
- `platform: str` – e.g. `"linkedin"` or `"twitter"`.


In [ ]:
from typing import TypedDict, List

class PostState(TypedDict):
    draft: str
    quality_score: int
    iteration: int
    max_iterations: int
    history: List[str]
    stop_reason: str
    feedback: str
    platform: str


## 2. Enhanced Evaluation Node – Smarter Scoring + Stop Reason

Implements:

- **Exercise 1:** Extra bonuses for `?` and time-horizon phrases.
- **Exercise 2:** Sets `stop_reason` when appropriate.


In [ ]:
def evaluate_post_node(state: PostState) -> PostState:
    text = state["draft"]
    lower = text.lower()

    score = 50

    # Length penalty
    if len(text) < 50:
        score -= 20

    # Call-to-action bonus
    if "comment" in lower or "let me know" in lower:
        score += 20

    # Learning/sharing bonus
    if "learning" in lower or "sharing" in lower:
        score += 10

    # Exercise 1 – Bonus for question mark
    if "?" in text:
        score += 5

    # Exercise 1 – Bonus for time horizon
    time_phrases = ["today", "this week", "this year", "over the next"]
    if any(phrase in lower for phrase in time_phrases):
        score += 5

    # Clamp
    score = max(0, min(100, score))
    state["quality_score"] = score

    # We don't set stop_reason here directly; we'll do that via the loop condition
    return state


## 3. Enhanced Improvement Node – Feedback + Platform Style

Implements:

- **Stretch Exercise 1:** `feedback`-aware improvements.
- **Stretch Exercise 2:** `platform`-aware length/style adjustments.


In [ ]:
def improve_post_node(state: PostState) -> PostState:
    draft = state["draft"]
    history = state["history"]
    feedback = state.get("feedback", "neutral")
    platform = state.get("platform", "linkedin")  # default

    history.append(draft)

    new_draft = draft
    lower = draft.lower()

    # Base improvement: add context if very short (but not too much on Twitter)
    if len(draft) < 50:
        extra = " I'm exploring this topic in depth and will share what I learn."
        if platform == "twitter":
            # Shorter extra for Twitter
            extra = " I'm exploring this topic and will share more soon."
        new_draft += extra

    # Call-to-action if missing (maybe shorter on Twitter)
    if "comment" not in lower and "let me know" not in lower:
        if platform == "twitter":
            new_draft += " Let me know your thoughts."
        else:
            new_draft += " Let me know your thoughts in the comments."

    # Feedback-based tweaks
    if feedback == "shorter":
        # If the draft is getting too long, trim it a bit
        if len(new_draft) > 220:  # pretend we want to stay under ~220 chars
            new_draft = new_draft[:217].rstrip() + "..."
    elif feedback == "more examples":
        if "for example" not in lower:
            new_draft += " For example, I'm focusing on small projects at work."    

    state["draft"] = new_draft
    state["history"] = history
    state["iteration"] = state["iteration"] + 1
    return state


## 4. Loop Condition with Stop Reason

We implement a loop condition helper that also sets `stop_reason` right before stopping.


In [ ]:
def should_continue(state: PostState) -> str:
    # Determine whether to continue or stop based on quality and iterations.
    quality = state["quality_score"]
    iteration = state["iteration"]
    max_iter = state["max_iterations"]

    if quality >= 70:
        state["stop_reason"] = "good_quality"
        return "stop"
    if iteration >= max_iter:
        state["stop_reason"] = "max_iterations_reached"
        return "stop"

    # Otherwise, continue
    return "continue"


## 5. Build the LangGraph App

We wire the enhanced nodes into a looping graph, same pattern as the teaching notebook but with extra fields.


In [ ]:
from langgraph.graph import StateGraph, END

# Create graph and add nodes
graph = StateGraph(PostState)

graph.add_node("evaluate_post", evaluate_post_node)
graph.add_node("improve_post", improve_post_node)

graph.set_entry_point("evaluate_post")

graph.add_conditional_edges(
    "evaluate_post",
    should_continue,
    {
        "continue": "improve_post",
        "stop": END,
    },
)

graph.add_edge("improve_post", "evaluate_post")

app = graph.compile()

## 6. Example Runs

Let's test different combinations of `feedback` and `platform` to see how the loop behaves.


In [ ]:
# Example 1 – Default LinkedIn-style draft
state1: PostState = {
    "draft": "Learning AI. Any tips?",
    "quality_score": 0,
    "iteration": 0,
    "max_iterations": 3,
    "history": [],
    "stop_reason": "",
    "feedback": "neutral",
    "platform": "linkedin",
}
app.invoke(state1)

In [ ]:
# Example 2 – Twitter-style draft with feedback "shorter"
state2: PostState = {
    "draft": "Learning about AI and data.",
    "quality_score": 0,
    "iteration": 0,
    "max_iterations": 5,
    "history": [],
    "stop_reason": "",
    "feedback": "shorter",
    "platform": "twitter",
}
app.invoke(state2)

In [ ]:
# Example 3 – Ask for more examples
state3: PostState = {
    "draft": "I'm learning AI at work.",
    "quality_score": 0,
    "iteration": 0,
    "max_iterations": 4,
    "history": [],
    "stop_reason": "",
    "feedback": "more examples",
    "platform": "linkedin",
}
app.invoke(state3)

You can inspect the returned `draft`, `quality_score`, `iteration`, `stop_reason`, and `history` to see how the loop evolved.

This notebook provides a reference implementation for the **looping pattern** in Lesson 5.
